In [63]:
import censusdata
import pandas as pd
import geopandas as gpd
import numpy as np
import warnings; warnings.simplefilter('ignore')

In [64]:
#censusdata.search('acs5', 2015, 'concept', 'citizenship')

In [65]:
#censusdata.printtable(censusdata.censustable('acs5', 2015, 'B05003'))

In [68]:
df = censusdata.download('acs5', 2019, censusdata.censusgeo([('state', '25'), ('county', '025'), ('tract', '*')]),
                             ['B05003_009E', 'B05003_011E', 'B05003_020E', 'B05003_022E'])

df = df.reset_index(drop=False)
df.rename(columns = {'index' : 'TRACT', 'B05003_009E' : 'Native Male 18+', 
                     'B05003_011E' : 'Naturalized Male 18+', 'B05003_020E' : 'Native Female 18+',
                     'B05003_022E' : 'Naturalized Female 18+'}, inplace = True)

for x in range(len(df)):
    df.loc[x, 'TRACT'] = df['TRACT'].unique()[x].geo[2][1]
    
df = df.astype({"TRACT": int})

sum_column = df['Native Male 18+'] + df['Native Female 18+'] + df['Naturalized Male 18+'] + df['Naturalized Female 18+']

df['CVAP'] = sum_column

df = df[['TRACT', 'CVAP']]

df

,TRACT,CVAP
0,802,4901
1,10600,2564
2,40100,1850
3,40300,3419
4,50400,1250
...,...,...
199,82100,3589
200,60200,1857
201,90400,2396
202,120500,2132


In [60]:
template = pd.read_csv('Precinct_Make_Up_Of_Census_Tracts.csv')
template = template.drop('Unnamed: 0', axis=1)

prct_age = pd.DataFrame(0, index=np.arange(len(template.columns[1:])),
                        columns=df.columns[1:])

prct_age.insert(0, 'WARD_PRECI', template.columns[1:])

for i in range(len(prct_age)):
    prct = prct_age['WARD_PRECI'].loc[i]
    temp = template[['TRACT', prct]]
    for n in range(len(temp)):
        if temp[prct].loc[n] != 0:
            dec = temp.loc[n, prct]
            filt = df['TRACT'] == temp.loc[n, 'TRACT']
            for c in df.columns[1:]:
                prct_age[c].iloc[i] += round(df[filt][c] * dec)

In [61]:
prct_age.to_csv('2015-2019_Precinct_CVAP.csv')

In [21]:
neighborhoods = gpd.read_file('Boston_Neighborhoods.shp')

props = pd.read_csv('Neighborhood_Make_Up_Of_Precincts.csv')

neigh_age_11_15 = pd.DataFrame(0, index=np.arange(len(neighborhoods['Name'])),
                        columns=prct_age.columns[1:])
neigh_age_11_15.insert(0, 'NEIGHBORHOOD', neighborhoods['Name'])

for i in range(len(neigh_age_11_15)):
    neighb = neigh_age_11_15['NEIGHBORHOOD'].loc[i]
    temp = props[['WARD_PRECI', neighb]]
    for n in range(len(temp)):
        if temp[neighb].loc[n] != 0:
            dec = temp[neighb].loc[n]
            filt = prct_age['WARD_PRECI'] == temp['WARD_PRECI'].loc[n]
            for c in prct_age.columns[1:]:
                try:
                    neigh_age_11_15[c].iloc[i] += round(prct_age[filt][c] * dec)
                except ValueError as v:
                    break

for c in neigh_age_11_15.columns:
    if c != 'NEIGHBORHOOD':
        neigh_age_11_15[c] = neigh_age_11_15[c].round().astype(int)

cols = list(neigh_age_11_15.columns)[2:]

for c in cols:
    for i in range(len(neigh_age_11_15)):
        neigh_age_11_15.loc[i, c] = round(neigh_age_11_15.loc[i, c]/neigh_age_11_15.loc[i, 'Total'], 3)

neigh_age_11_15

,NEIGHBORHOOD,CVAP
0,Roslindale,20492
1,Jamaica Plain,30927
2,Mission Hill,10307
3,Longwood,5161
4,Bay Village,1058
5,Leather District,358
6,Chinatown,1838
7,North End,6218
8,Roxbury,33324
9,South End,18020
